In [1]:
#encoding: utf-8
from bs4 import BeautifulSoup
import unidecode
import requests
import pandas as pd
import folium
from geopy.geocoders import Nominatim
import math  
    
request = requests.get('https://api.covid19.finspect.me/brcovid19/map')
df=pd.DataFrame(request.json())
address = 'Brazil'

request1 = requests.get('https://api.covid19.finspect.me/brcovid19/all')
df1=pd.DataFrame(request1.json())

geolocator = Nominatim(user_agent="CovidBrazil", timeout=10)
location = geolocator.geocode(address)
latitude = -16.703549 #location.latitude
longitude =-49.265704 #location.longitude
 
map_Covid = folium.Map(location=[latitude, longitude], zoom_start=4)

df["lat"] = pd.to_numeric(df["lat"])
df["lng"] = pd.to_numeric(df["lng"])   
df_describe=pd.DataFrame(df['qtd_confirmation'].describe())
url = 'https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data'
state_geo = f'{url}/brazil-states.geojson'

map_Covid.choropleth(
    geo_data=state_geo,
    data=df,
    columns=['name', 'qtd_confirmation'],
    key_on='feature.properties.name',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    threshold_scale=[df['qtd_confirmation'].min(),df['qtd_confirmation'].max()/16,df['qtd_confirmation'].max()/8, df['qtd_confirmation'].max()/4, df['qtd_confirmation'].max()/2, df['qtd_confirmation'].max()+1],
    legend_name='Numeros de casos'
)


latitudes = df.lat
longitudes = df.lng
labels = df["qtd_confirmation"].astype(str) + ", " + df['name'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')


for lat, lng, label in zip(latitudes, longitudes, labels):
    folium.Marker([lat, lng], popup=label).add_to(map_Covid)
    
#display map
map_Covid    



In [ ]:
indexFile=r'C:\Users\%Username%\Desktop\projects\covid_19\index.html'
map_Covid.save(indexFile)

html = open(indexFile).read()
soup = BeautifulSoup(html)


new_h1 = soup.new_tag('h1')
new_h1.string="Brazil - Covid-19 cases split by States"
soup.html.insert(0, new_h1)

new_div = soup.new_tag('div')
soup.h1.wrap(new_div)

textHtml = 'Cases: {} / Deaths: {} / Lethality: {} / Last update: {}.'.format(df1.total_confirmation[0], df1.total_death[0],df1.total_leathality[0],df1.dt_updated[0] )

for article in soup.find_all('h1'):
    new_tag = soup.new_tag("h5")
    new_tag.append(textHtml)
    article.insert_after(new_tag)
    
textHtml = "Source: https://covid.finspect.me/.        Created by Cesar Kagohara"
for article in soup.find_all('h5'):
    # create a new tag
    new_tag = soup.new_tag("h6")
    new_tag.append(textHtml)
    article.insert_after(new_tag)

with open(indexFile, "w", encoding='utf-8') as file:
    file.write(str(soup))